# Image Processing SS 20 - Assignment - 04

### Deadline is 20.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import random
from io import BytesIO
from PIL import Image

pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

# Exercise 1 - Qualify sharpness and noise - 5 Points

Determine the noise and sharpness in the images. Plot image number vs noise

Please download sample picture from [here](http://sipi.usc.edu/database/misc.zip) and place them next to your assignment (inside the misc folder).

In [ ]:
# Load the pictures here
sample_images = []
direc = 'misc/' # directory of the sample pictures relative to your notebook 
for number in [1,3,5,6]:
    sample_images.append(
        np.array(Image.open(direc+'4.2.0'+str(number)+'.tiff'))
    )
for name in ['house']:
    sample_images.append(
        np.array(Image.open(direc+name+'.tiff'))
    )

In [ ]:
def qualify_noise(img):
    """Qualify the noise based on the std of a gaussian model.
       You may find a window that is contant in the images.
    """
    # your code here
    img = img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114 #convert RGB to Grey
    var=[]
    hoehe = img.shape[0] 
    laenge = img.shape[1]
    size_search_window=11
    width_search=size_search_window//2
    search_step=4
    Mittelpunkt_x = np.array(range(width_search-1,hoehe-width_search,search_step)) #Koordinaten von Mittelpunkten
    Mittelpunkt_y = np.array(range(width_search-1,laenge-width_search,search_step))
    for i in  range(len(Mittelpunkt_x)):
        for j in  range(len(Mittelpunkt_y)) :
            search_fenster = img[Mittelpunkt_x[i]-(width_search-1):Mittelpunkt_x[i] + width_search + 1,Mittelpunkt_y[j]-(width_search-1):Mittelpunkt_y[j]+width_search+1]
            fenster_var=np.var(search_fenster)
            var.append(fenster_var)             #using 11*11 window to find the minimal variance
    return np.sqrt(min(var))

plt.bar(range(len(sample_images)), [qualify_noise(i) for i in sample_images])

In [ ]:
def qualify_sharpness(img):
    """Qualify the sharpness based on the average pixel differences."""
    # your code here
    img_grey = img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114 #convert RGB to Grey
    diff_x=np.zeros_like(img)
    diff_y=np.zeros_like(img)
    for i in range(img_grey.shape[0]-1):
        for j in range(img_grey.shape[1]-1):
            diff_x[i,j]=img_grey[i+1,j]-img_grey[i,j]
            diff_y[i,j]=img_grey[i,j+1]-img_grey[i,j]
    return np.mean(np.sqrt(diff_x**2+diff_y**2))
plt.bar(range(len(sample_images)), [qualify_sharpness(i) for i in sample_images])

Does the result match your expectations? If not what processing step can be done?

For noise qualification : for a more accurate result,we have to reduce the sliding step, which will lead to low efficiency. So better strategy is using gradient descent to find the minimal Variance instead of search through the whole image.

For sharpness qualification : the distinction of first order of difference of image is not significant. Second Order of difference can be considered. 

# Exercise 2 - SSIM JPEG Compression - 5 Points

In [ ]:
def jpeg_enocde(img, quality):
    pil_img = Image.fromarray(img)
    buffer = BytesIO()
    pil_img.save(buffer, "JPEG", quality=quality)
    return buffer

def jpeg_decode(buffer):
    img = Image.open(buffer)
    return np.array(img)

def jpeg_quality_filter(img, quality):
    as_jpeg = jpeg_enocde(img, quality)
    return jpeg_decode(as_jpeg)

In [ ]:
images_for_jpeg = sample_images[2::]
len(images_for_jpeg)

In [ ]:
images10 = [jpeg_quality_filter(img, 10) for img in images_for_jpeg]
images50 = [jpeg_quality_filter(img, 50) for img in images_for_jpeg]
images80 = [jpeg_quality_filter(img, 80) for img in images_for_jpeg]

In [ ]:
from scipy.ndimage import gaussian_filter

def ssim(img, filtered_img):

    K1 = 0.01
    K2 = 0.03
    sigma = 1.5
    
    #using gaussian sliding windows 
    filter_func = gaussian_filter
    filter_args = {'sigma': sigma}


    # ndimage filters need floating point data
    X=img
    Y=filtered_img
    X = X.astype(np.float64)
    Y = Y.astype(np.float64)
    
    # compute (weighted) means
    mux = filter_func(X, **filter_args)
    muy = filter_func(Y, **filter_args)

    # compute (weighted) variances and covariances
    muxx = filter_func(X * X, **filter_args)
    muyy = filter_func(Y * Y, **filter_args)
    # calculate E(XY)
    muxy = filter_func(X * Y, **filter_args)
    # sigma_x^2 = E(x^2)-E(x)^2
    vax = muxx - mux * mux
    # sigma_y^2 = E(y^2)-E(y)^2
    vay = muyy - muy * muy
    # cov(x,y) = E(xy)-E(x)E(y)
    vaxy = muxy - mux * muy
    
    dynamic_range=255
    # Formula from the paper
    C1 = (K1 * dynamic_range) ** 2
    C2 = (K2 * dynamic_range) ** 2
    
    # Formula of SSIM
    S=((2 * mux * muy + C1)*(2 * vaxy + C2))/((mux** 2 + muy ** 2 + C1)*(vax + vay + C2))

    return S.mean()

for i, img in enumerate(images_for_jpeg):
    print(i)
    compressed_images = [images10[i], images50[i], images80[i]]
    plt.bar(range(len(compressed_images)),
             [ssim(img, comp) for comp in compressed_images])
    plt.show()



The Gaussian Sliding Windows is referenced from :https://github.com/elifesciences-publications/Bhatia_et_al_2019/blob/master/code/external/scikit-image-0.14.1/skimage/measure/_structural_similarity.py

In [ ]:
#we are not sure, whether we can import other module to solve this problem,so we write a backup 
""""def ssim(img, filtered_img):
   
    # your code
    img = img[:,:,0]*0.299 + img[:,:,1]*0.587 + img[:,:,2]*0.114 #convert RGB to Grey
    filtered_img=filtered_img[:,:,0]*0.299 + filtered_img[:,:,1]*0.587 + filtered_img[:,:,2]*0.114 #convert RGB to Grey
    k1=0.01
    k2=0.03
    L=255
    c1=(k1*255)**2
    c2=(k2*255)**2
    SSIM=[]
    hoehe = img.shape[0] 
    laenge = img.shape[1]
    size_search_window=11
    width_search=size_search_window//2
    Mittelpunkt_x = np.array(range(width_search-1,hoehe-width_search,size_search_window)) #Koordinaten von Mittelpunkten
    Mittelpunkt_y = np.array(range(width_search-1,laenge-width_search,size_search_window))
    for i in  range(len(Mittelpunkt_x)):
        for j in  range(len(Mittelpunkt_y)) :
            search_fenster_1 = img[Mittelpunkt_x[i]-(width_search-1):Mittelpunkt_x[i] + width_search + 1,Mittelpunkt_y[j]-(width_search-1):Mittelpunkt_y[j]+width_search+1]
            search_fenster_2 = filtered_img[Mittelpunkt_x[i]-(width_search-1):Mittelpunkt_x[i] + width_search + 1,Mittelpunkt_y[j]-(width_search-1):Mittelpunkt_y[j]+width_search+1]
            mux=np.mean(search_fenster_1)
            muy=np.mean(search_fenster_2)
            muxy=np.mean(search_fenster_2*search_fenster_1)
            sigmaxy=muxy-mux*muy
            #sigmaxy=np.cov(search_fenster_1,search_fenster_2)
            sigmax=np.var(search_fenster_1)
            sigmay=np.var(search_fenster_2)
            A1, A2, B1, B2 = ((2 *mux * muy + c1,
                       2 * sigmaxy + c2,
                       mux ** 2 + muy ** 2 + c1,
                       sigmax + sigmay + c2))
            D = B1 * B2
            S = (A1 * A2) / D
            #ssim=(2*mux*muy+c1)*(2*sigmaxy+c2)/((mux*2+muy**2+c1)*(sigmax**2+sigmay**2+c2))
            SSIM.append(S)
    return np.mean(SSIM)

for i, img in enumerate(images_for_jpeg):
    print(i)
    compressed_images = [images10[i], images50[i], images80[i]]
    plt.bar(range(len(compressed_images)),
             [ssim(img, comp) for comp in compressed_images])
    plt.show()